In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import joblib

In [2]:
data = pd.read_csv('ielts_writing_dataset.csv')

X = data[['Question', 'Essay']]
y = data['Overall']



In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['Question'] + ' ' + X_train['Essay'])
X_test_tfidf = vectorizer.transform(X_test['Question'] + ' ' + X_test['Essay'])

In [4]:
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 'auto'], 'kernel': ['linear', 'rbf', 'poly']}
grid_search = GridSearchCV(SVR(), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')


Best Parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [5]:
final_svm_model = SVR(**best_params)
final_svm_model.fit(X_train_tfidf, y_train)
cross_val_scores = cross_val_score(final_svm_model, X_train_tfidf, y_train, cv=5)
print(f'Cross-Validation Scores: {cross_val_scores}')

Cross-Validation Scores: [0.18648249 0.26696899 0.22179538 0.18108662 0.31817224]


In [6]:
y_pred = final_svm_model.predict(X_test_tfidf)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.7843070417319679


In [7]:
joblib.dump(final_svm_model, 'svm_model.joblib')
print('Model saved successfully.')

Model saved successfully.


In [8]:
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')


['tfidf_vectorizer.joblib']

In [9]:
loaded_model = joblib.load('svm_model.joblib')


In [10]:
new_input = pd.DataFrame({'Question': ['Newspapers and books are outdated. Why do some people believe this? What is your opinion?'], 'Essay': ['The twenty-first century illustrates an increasing number of people are relying on new forms of technology. A possible consequence of this is that traditional media such as books and newspapers are not common, and these documents are considered to be outdated. Personally, I disagree with this point of view, books and newspapers should be a basic way of saving knowledge and traditional culture.The principal reason that some people give this view is discovering information in the global from the internet or multimedia such as television, podcast channel is much quicker and easier. For instance, if people use Google or another search engine, they can instantly get the latest news. The fact that technology gives more convenience in browsing information, but most of these are the fast-moving news, and when an individual wants to research or directly interact with the words, in reality, the traditional platform such as research reports, academic notebooks are more essential than the high-technical platform.Furthermore, reading print books and newspapers can be better for the brain and health. Because readers of print books absorb and remember more of the plot than readers of e-books or digital magazines. For example, many jobs require people to stare at a computer screen all day, and it will give their eyes a break. Electronic books can cause screen fatigue, which may lead to blurred vision, dryness, and irritation, with print books, those problems do not occur any more. In addition, researchers believe that if readers have a books or personal library at home, they will be more likely to link to higher academic achievement.E-books and internet platforms would be more convenient than traditional books. But the important thing is reading on paper books can enrich the entire experience, and readers will not be got sidetracked more easily than digital readers. Thus, traditional reading will not be outdated.']})
new_input_tfidf = vectorizer.transform(new_input['Question'] + ' ' + new_input['Essay'])
predicted_marks = final_svm_model.predict(new_input_tfidf)
print(f'Predicted Marks for New Input: {predicted_marks[0]}')

Predicted Marks for New Input: 7.4297777343919265
